In [ ]:
import json, os
from pathlib import Path

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_trades_from_db, load_backtest_data, load_backtest_stats
from freqtrade.data.history import load_pair_history
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plugins.pairlistmanager import PairListManager
from freqtrade.exceptions import ExchangeError, OperationalException
from freqtrade.exchange import Exchange
from freqtrade.resolvers import ExchangeResolver, StrategyResolver
from freqtrade.state import RunMode

import numpy as np
import pandas as pd
import random
from collections import deque

import nest_asyncio
nest_asyncio.apply()

configs=["cryptofrog.config.json"]

ft_config = Configuration.from_files(files=configs)
ft_exchange = ExchangeResolver.load_exchange(ft_config['exchange']['name'], config=ft_config, validate=True)
ft_pairlists = PairListManager(ft_exchange, ft_config)
ft_dataprovider = DataProvider(ft_config, ft_exchange, ft_pairlists)

data_location = Path(ft_config['user_data_dir'], 'data', 'binance')
backtest_dir = Path(ft_config['user_data_dir'], 'backtest_results')

# Load strategy using values set above
strategy = StrategyResolver.load_strategy(ft_config)
strategy.dp = ft_dataprovider

# Generate buy/sell signals using strategy
timeframe = "5m"

backtest = False

if ft_config["timeframe"] is not None:
    timeframe = ft_config["timeframe"]
    print("Using config timeframe:" , timeframe)
elif strategy.timeframe is not None:
    timeframe = strategy.timeframe
    print("Using strategy timeframe:" , timeframe)
else:
    print("Using default timeframe:" , timeframe)

def do_analysis(pair, strategy, timeframe, only_backtest=False, hist_and_backtest=True):
    if only_backtest is True:
        print("Loading backtest data...")
        trades = load_backtest_data(backtest_dir)
    else:
        print("Loading historic data...")
        candles = load_pair_history(datadir=data_location,
                                    timeframe=timeframe,
                                    pair=pair,
                                    data_format = "json",
                                    )

        # Confirm success
        print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
        df = strategy.analyze_ticker(candles, {'pair': pair})
        
        if hist_and_backtest is True:
            print("Loading backtest trades data...")
            trades = load_backtest_data(backtest_dir)            
        else:
            # Fetch trades from database
            print("Loading DB trades data...")
            trades = load_trades_from_db(ft_config['db_url'])

        print(f"Generated {df['buy'].sum()} buy / {df['sell'].sum()} sell signals")
        data = df.set_index('date', drop=False)
        # print(data.info())
        # print(data.tail())
        # print(data["buy"].dropna())
        return (data, trades)
    
    return (pd.Dataframe(), trades)

def do_plot(pair, data, trades, plot_config=None):
    from freqtrade.plot.plotting import generate_candlestick_graph
    import plotly.offline as pyo

    # Filter trades to one pair
    trades_red = trades.loc[trades['pair'] == pair].copy()

    # Limit graph period to your BT timerange
    data_red = data['2021-04-01':'2021-04-20']

    plotconf = strategy.plot_config
    if plot_config is not None:
        plotconf = plot_config
    
    # Generate candlestick graph
    graph = generate_candlestick_graph(pair=pair,
                                       data=data_red,
                                       trades=trades_red,
                                       plot_config=plotconf
                                       )

    pyo.iplot(graph, show_link = False)

In [ ]:
# example inline plot config to override strat
plot_config = {
    'main_plot': {
    },
    'subplots': {
        'STR' :{
            'mastreak': {'color': 'black'},
        },
        'KAMA' :{
            'kama': {'color': 'red'},
        },
        'RMI' :{
            'rmi': {'color': 'blue'},
        },
        'MP' :{
            'mp': {'color': 'green'},
        },        
    }
}

## set the pairlist you want to plot
pairlist = ["WIN/USDT", "SC/USDT"]

## don't do this for more than a couple of pairs and for a few days otherwise Slowness Will Occur
for i in pairlist:
    (data, trades) = do_analysis(i, strategy, timeframe)
    do_plot(i, data, trades) #, plot_config=plot_config)